In [78]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
import numpy as np

from utils.metrics import ndcg_scorer

In [79]:
path = '../data/processed/'
train_users = pd.read_csv(path + 'processed_train_users.csv').head(100000)

In [80]:
y_train = train_users['country_destination']
train_users.drop('country_destination', axis=1, inplace=True)
train_users.drop('id', axis=1, inplace=True)
train_users = train_users.fillna(-1)
x_train = train_users.values
label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [81]:
xgb = XGBClassifier(
    max_depth=10,
    learning_rate=0.3,
    n_estimators=10,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=1,
    colsample_bytree=1,
    colsample_bylevel=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    missing=None,
    silent=True,
    nthread=-1,
    seed=42
)

kf = KFold(len(x_train), n_folds=10, random_state=42)

In [83]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('feature_selection', SelectFromModel(RandomForestClassifier())),
  ('classification', xgb)
])

In [ ]:
pipe_score = cross_val_score(clf, x_train, encoded_y_train, cv=kf, scoring=ndcg_scorer)

In [ ]:
xgb_score = cross_val_score(xgb, x_train, encoded_y_train, cv=kf, scoring=ndcg_scorer)